In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18316034838430988270]

In [ ]:
!cat /proc/meminfo

MemTotal:       13302920 kB
MemFree:        10454344 kB
MemAvailable:   12395312 kB
Buffers:          122276 kB
Cached:          1952824 kB
SwapCached:            0 kB
Active:          1108272 kB
Inactive:        1493904 kB
Active(anon):     489664 kB
Inactive(anon):      428 kB
Active(file):     618608 kB
Inactive(file):  1493476 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:             15648 kB
Writeback:             0 kB
AnonPages:        527064 kB
Mapped:           380448 kB
Shmem:              1148 kB
KReclaimable:     140928 kB
Slab:             185116 kB
SReclaimable:     140928 kB
SUnreclaim:        44188 kB
KernelStack:        4800 kB
PageTables:         6784 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651460 kB
Committed_AS:    3230548 kB
VmallocTotal:   34359738367 kB
VmallocUsed:        7188 kB
VmallocChunk:          0 kB
Percpu:          

# LOAD DATASET MNIST

In [10]:
pip install python_mnist

In [11]:
 pip install scikit-optimize

     |████████████████████████████████| 101 kB 3.4 MB/s 


In [12]:
from mnist import MNIST
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model.sag import sag_solver
from sklearn import svm
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats
from numpy import mean
from numpy import std
import skopt
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.sag module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [13]:
  pip install scikit-optimize

In [14]:

# # report scikit-optimize version number
import skopt
print('skopt %s' % skopt.__version__)

skopt 0.8.1


In [15]:
pip install scikit_learn

In [16]:
pip install "scikit_learn==0.22.2.post1"

#LOAD THE DATA

In [66]:

#Load dataset
cancer = datasets.load_breast_cancer()

In [67]:
# print the names of the 13 features
print("Features: ", cancer.feature_names)

# print the label type of cancer('malignant' 'benign')
print("Labels: ", cancer.target_names)

Features:  ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
Labels:  ['malignant' 'benign']


In [71]:
# print data(feature)shape
cancer.data.shape,cancer.target.shape

((569, 30), (569,))

In [20]:

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.3,random_state=109) # 70% training and 30% test



In [22]:

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [23]:
X_train.shape,  X_test.shape

((398, 30), (171, 30))

#GRID SEARCH WITH SVM

In [55]:
#svm with default hyperparameters
import time
start = time.time()
clf = SVC( C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0)
clf.fit(X_train,y_train)
scores = cross_val_score(clf, X_train,y_train, cv=5,scoring='accuracy')
print("Accuracy:"+ str(scores.mean()))
end = time.time()
print(f"Runtime of the program is {end - start}")


Accuracy:0.9647784810126583
Runtime of the program is 0.0465848445892334


# svm with tune hyperparameters using grid search

In [54]:
import time
start = time.time()
kernel = ['rbf','poly','linear','sigmoid']
C=[1.0,1.1,5.4,170,1001]
gamma = [0.7001, 0.008, 0.001, 3.1, 1, 1.3, 5]
high_accuracy = 0.0
hypers = []
for i in C:
  for j in gamma:
    for k in kernel:
      hypers.append((i,j,k))

    
print('all combinations :',hypers)
params = {
  'params': '',
  'score': 0.
}
for k in range(len(hypers)):
  model = SVC(C = hypers[k][0],gamma = hypers[k][1],kernel=hypers[k][2])
  model.fit( X_train, y_train )
  Y_pred = model.predict( X_test)
  classified_correct = 0

  for total in range(np.size(Y_pred)):
    if y_test[total] == Y_pred[total]:
      classified_correct = classified_correct+1
  accuracy =(classified_correct/total)*100

  if params['score'] < accuracy:
    params['score'] = accuracy
    params['params'] = hypers[k]
end = time.time()
print(f"Runtime of the program is {end - start}")




all combinations : [(1.0, 0.7001, 'rbf'), (1.0, 0.7001, 'poly'), (1.0, 0.7001, 'linear'), (1.0, 0.7001, 'sigmoid'), (1.0, 0.008, 'rbf'), (1.0, 0.008, 'poly'), (1.0, 0.008, 'linear'), (1.0, 0.008, 'sigmoid'), (1.0, 0.001, 'rbf'), (1.0, 0.001, 'poly'), (1.0, 0.001, 'linear'), (1.0, 0.001, 'sigmoid'), (1.0, 3.1, 'rbf'), (1.0, 3.1, 'poly'), (1.0, 3.1, 'linear'), (1.0, 3.1, 'sigmoid'), (1.0, 1, 'rbf'), (1.0, 1, 'poly'), (1.0, 1, 'linear'), (1.0, 1, 'sigmoid'), (1.0, 1.3, 'rbf'), (1.0, 1.3, 'poly'), (1.0, 1.3, 'linear'), (1.0, 1.3, 'sigmoid'), (1.0, 5, 'rbf'), (1.0, 5, 'poly'), (1.0, 5, 'linear'), (1.0, 5, 'sigmoid'), (1.1, 0.7001, 'rbf'), (1.1, 0.7001, 'poly'), (1.1, 0.7001, 'linear'), (1.1, 0.7001, 'sigmoid'), (1.1, 0.008, 'rbf'), (1.1, 0.008, 'poly'), (1.1, 0.008, 'linear'), (1.1, 0.008, 'sigmoid'), (1.1, 0.001, 'rbf'), (1.1, 0.001, 'poly'), (1.1, 0.001, 'linear'), (1.1, 0.001, 'sigmoid'), (1.1, 3.1, 'rbf'), (1.1, 3.1, 'poly'), (1.1, 3.1, 'linear'), (1.1, 3.1, 'sigmoid'), (1.1, 1, 'rbf'),

In [ ]:
params

{'params': (1.0, 0.7001, 'linear'), 'score': 99.41176470588235}

#GRID SEACRH WITH LOGISTIC REGRESSION

In [48]:
#baseline model
import time
start = time.time()

clf = LogisticRegression()
clf.fit(X_train, y_train)
scores = cross_val_score(clf, X_train, y_train, cv=3,scoring='accuracy')
print("Accuracy:"+ str(scores.mean()))

end = time.time()
print(f"Runtime of the program is {end - start}")

Accuracy:0.9698108908635225
Runtime of the program is 0.05045628547668457


In [49]:
#tuning hyperparameters
start = time.time()

clf = LogisticRegression()
params = {'C':[0.001,.009,0.01,.09,1,5,10,25],'penalty': ['l1', 'l2'],'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
grid= GridSearchCV(clf, params,cv=3,scoring = 'accuracy')
grid.fit(X_train, y_train)
print(grid.best_params_)
print("Accuracy:"+ str(grid.best_score_))

end = time.time()
print(f"Runtime of the program is {end - start}")

{'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Accuracy:0.9773296878560037
Runtime of the program is 1.4455034732818604


# GRID SEARCH WITH KNN

In [50]:
#knn by defaut hyperparameters
start = time.time()
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
scores = cross_val_score(knn, X_train, y_train, cv=3,scoring='accuracy')
print("Accuracy:"+ str(scores.mean()))
end = time.time()
print(f"Runtime of the program is {end - start}")


Accuracy:0.9522670312143996
Runtime of the program is 0.04008936882019043


In [51]:
#knn by tunning hyperparamenters
start = time.time()
params_grid = {"n_neighbors": range(1, 50),'p':[1,2]}
grid_search = GridSearchCV(knn, params_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
print("Accuracy:"+ str(grid_search.best_score_))
end = time.time()
print(f"Runtime of the program is {end - start}")


{'n_neighbors': 8, 'p': 2}
Accuracy:0.9597668413457887
Runtime of the program is 2.83908748626709
